In [ ]:
!pip install tensorly


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from queue import PriorityQueue
import tensorly
from tensorly.decomposition import parafac
import math
from tensorly.metrics import regression

import random


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Function that selecteds random samples from all three modes and returns them with their corresponding indices
def selectRandomSamples(A1slice,A2slice,A3slice,s1,s2,s3):

    #moi along I:
    pqI = PriorityQueue()
    sum_weight = 0
    for i in range(0,A1slice.shape[0]):
        sum_weight = 0
        for j in range(0,A1slice.shape[1]):
            for k in range(0,A1slice.shape[2]):
                #print("c:",A1slice[i][j][k])
                sum_weight += A1slice[i][j][k]
        pqI.put((-sum_weight,i))



    selectedI = np.zeros([s1,A1slice.shape[1],A1slice.shape[2]])
    selectedindxI = np.zeros(s1)
    i = 0
    while not pqI.empty():
        next_item = pqI.get()
        if i < s1:
            selectedI[i] = A1slice[next_item[1]]
            selectedindxI[i] = next_item[1]
        i += 1






    #moi along J:
    pqJ = PriorityQueue()
    sum_weight = 0

    for i in range(0,A2slice.shape[0]):
        sum_weight = 0
        for j in range(0,A2slice.shape[1]):
            for k in range(0,A2slice.shape[2]):
                #print("c:",A1slice[i][j][k])
                sum_weight += A2slice[i][j][k]
        pqJ.put((-sum_weight,i))

    selectedJ = np.zeros([s2,A2slice.shape[1],A2slice.shape[2]])
    selectedindxJ = np.zeros(s2)
    i = 0

    while not pqJ.empty():
        next_item = pqJ.get()
        if i < s2:
            selectedJ[i] = A2slice[next_item[1]]
            selectedindxJ[i] = next_item[1]
        i += 1



    #moi along K:
    pqK = PriorityQueue()
    sum_weight = 0
    for i in range(0,A3slice.shape[0]):
        sum_weight = 0
        for j in range(0,A3slice.shape[1]):
            for k in range(0,A3slice.shape[2]):
                #print("c:",A1slice[i][j][k])
                sum_weight += A3slice[i][j][k]
        pqK.put((-sum_weight,i))

    selectedK = np.zeros([s3,A3slice.shape[1],A3slice.shape[2]])
    selectedindxK = np.zeros(s3)

    i = 0

    while not pqK.empty():
        next_item = pqK.get()
        if i < s3:
            selectedK[i] = A3slice[next_item[1]]
            selectedindxK[i] = next_item[1]
        i += 1


    return selectedI,selectedJ,selectedK,selectedindxI,selectedindxJ,selectedindxK

In [ ]:
#Function that builds the Xs tensor
#Xs tensor is made by combining all selected samples(from selectRandomSamples function) from all three slices
#The F parameter is a tensor with new incoming data
#F is set to None in the first iteration i.e for the initial tensor(before any new data has arrived)
#If new data are sent as parameter during function call, it adds the new data slices to Xs along with the older randomly selected samples
def buildXs(selectedI,selectedJ,selectedK,F):
    if F == None:
        Xs = np.zeros([(selectedI.shape[0]+selectedJ.shape[0]+selectedK.shape[0]),max(selectedI.shape[1],selectedJ.shape[1],selectedK.shape[1]),max(selectedI.shape[2],selectedJ.shape[2],selectedK.shape[2])])
    else:
        fixed1 = F[0]
        fixed2 = F[1]
        fixed3 = F[2]
        Xs = np.zeros([(selectedI.shape[0]+selectedJ.shape[0]+selectedK.shape[0]+fixed1.shape[0]+fixed2.shape[0]+fixed3.shape[0]),max(selectedI.shape[1],selectedJ.shape[1],selectedK.shape[1],fixed1.shape[1],fixed2.shape[1],fixed3.shape[1]),max(selectedI.shape[2],selectedJ.shape[2],selectedK.shape[2],fixed1.shape[2],fixed2.shape[2],fixed3.shape[2])])

    l=0
    m=0
    n=0

    for i in range(0,selectedI.shape[0]):
        for j in range(0,selectedI.shape[1]):
            for k in range(0,selectedI.shape[2]):
                Xs[i][j][k] = selectedI[i][j][k]
                l=i
                m=j
                n=k
                
    # Add new incoming slices
    if F != None:
        for i in range(0,fixed1.shape[0]):
            l+=1
            for j in range(0,fixed1.shape[1]):
                for k in range(0,fixed1.shape[2]):
                    Xs[l][j][k] = fixed1[i][j][k]

    for i in range(0,selectedJ.shape[0]):
        l+=1
        for j in range(0,selectedJ.shape[1]):
            for k in range(0,selectedJ.shape[2]):
                Xs[l][j][k] = selectedJ[i][j][k]
    
    if F != None:
        for i in range(0,fixed2.shape[0]):
            l+=1
            for j in range(0,fixed2.shape[1]):
                for k in range(0,fixed2.shape[2]):
                    Xs[l][j][k] = fixed2[i][j][k]

    for i in range(0,selectedK.shape[0]):
        l+=1
        for j in range(0,selectedK.shape[1]):
            for k in range(0,selectedK.shape[2]):
                Xs[l][j][k] = selectedK[i][j][k]
    if F != None:
        for i in range(0,fixed3.shape[0]):
            l+=1
            for j in range(0,fixed3.shape[1]):
                for k in range(0,fixed3.shape[2]):
                    Xs[l][j][k] = fixed3[i][j][k]



    return Xs


In [ ]:
#This function returns a summary of the tensor A
#Summary of a tensor is a tensor Xs which has all the randomly selected samples from all trhee modes
#The summary may or may not include the new incoming data slices depending on whether it is the initial iteration or not

def summary(A,s,fixed = None):
    #determine sample sizes
    I = A.shape[0]
    J = A.shape[1]
    K = A.shape[2]
    s1 = math.ceil(I/s[0])
    s2 = math.ceil(J/s[1])
    s3 = math.ceil(K/s[2])

    
    #Slice input matrix A along all dimensions
    A1slice = np.array(tf.slice(A, [0, 0, 0], [tf.shape(A)[0],tf.shape(A)[1] , tf.shape(A)[2]]))
    print("i mode slicing done")
    

    A2slice = np.zeros([A.shape[1],A.shape[0],A.shape[2]])
    for i in range(0,tf.shape(A)[1]):
        col = tf.slice(A, [0, i, 0], [tf.shape(A)[0],1 , tf.shape(A)[2]])
        A2slice[i] = col[:,0]
    print("j mode slicing done")
    
    A3slice = np.zeros([A.shape[2],A.shape[0],A.shape[1]])
    for i in range(0,tf.shape(A)[2]):
        col = tf.slice(A, [0, 0, i], [tf.shape(A)[0],tf.shape(A)[1] , 1])
        A3slice[i] = col[:,:,0]
    print("k mode slicing done")
    
    
    #Choose random samples along modes 1,2,3
    selectedI,selectedJ,selectedK,selectedindxI,selectedindxJ,selectedindxK = selectRandomSamples(A1slice,A2slice,A3slice,s1,s2,s3)
  
    #Combine samples
    if(fixed == None):
        Xs = buildXs(selectedI,selectedJ,selectedK,F = None)
    else:
        Xs = buildXs(selectedI,selectedJ,selectedK,F = fixed)

    
    return Xs,selectedI,selectedJ,selectedK,selectedindxI,selectedindxJ,selectedindxK
    

In [ ]:
# Build Xnew is used in place of the original tensor when new incoming slices have come
# When no incoming slices have come yet, and tensor A is at it's initial state, a set of randomly selected samples are selected and saved
# When new incoming data has arrived, the prevous samples selected when input tensor was at it's initial state are passed as parameters to this function
# Xnew is built by adding 2 things:
#     1. Adding the previously selected samples one after another using buildXs to create tensor Xslices
#     2. Adding the new incoming data slices.(Let use assume that the new data slices are only added along mode three for this code)
# Xnew is then returned
def buildXnew(Xold,F,mode1,mode2,mode3):
    #Xslices tensor shape:
    m = mode1.shape[0] 
    n = mode2.shape[0] 
    p = mode3.shape[0]
    
    #Build Xslices tensor with mode1,2 and 3 selected slices from previoud tensor

    """for i in range(0,m):
        for j in range(0,n):
            for k in range(0,p):
                Xslices[i][j][k] = Xold[int(mode1[i])][int(mode2[j])][int(mode3[k])]"""
    
    Xslices = buildXs(mode1,mode1,mode3,F = None)
                
    
    
    
    
    #Merge Xslices and new incoming slices
    m = max(Xslices.shape[0],F.shape[0])
    n = max(Xslices.shape[1],F.shape[1])
    p = Xslices.shape[2] + F.shape[2]
    # Here, only p is the added value of shape(2) of Xslices and F because the slices are added along the third dimension
    new = 0
    Xnew = np.zeros([m,n,p])

    
    for i in range(0,m):
        for j in range(0,n):
            for k in range(0,p):
                if k < Xslices.shape[2] and i < Xslices.shape[0] and j < Xslices.shape[1]:
                    Xnew[i][j][k] = Xslices[i][j][k]
                elif k >= Xslices.shape[2] and new < F.shape[2] and i < F.shape[0] and j < F.shape[1]:
                    Xnew[i][j][k] = F[i][j][new]
                    new+=1
            new = 0
                
        
   
    
                
                
    return Xnew
    
                

In [ ]:
def merge(As,Bs,Cs,r, rank,selectedI,selectedJ, selectedK ,Aold=None,Bold=None,Cold=None):
    usedA =[]
    if r ==0:
        A = As
        B = Bs
        C = Cs
    else:
        vA=np.zeros([Aold.shape[0],1])
        vB=np.zeros([Bold.shape[0],1])
        vC=np.zeros([Cold.shape[0],1])
        
        for i in range(0,rank):
            for j in range(0,rank):
                vA[j] = np.dot(Aold[:,i] , As[:,j])
                vB[j] = np.dot(Bold[:,i] , Bs[:,j])
                vC[j] = np.dot(Cold[:,i] , Cs[:,j])
                
            idx = np.argsort(vA)
            idx = idx[::-1]
            
            ii=0
            while(idx[ii] in usedA):
                ii= ii+1
            
            Id = idx[ii]
            usedA.append(Id)
            
            for k in range(0,As.shape[0]):
                if As[k][Id] == 0:
                    As[k][Id] = As[k][Id] + Aold[k][i]
                    
            for k in range(0,Bs.shape[0]):
                if Bs[k][Id] == 0:
                    Bs[k][Id] = Bs[k][Id] + Bold[k][i]
                    
            for k in range(0,Cs.shape[0]):
                if Cs[k][Id] == 0:
                    Cs[k][Id] = Cs[k][Id] + Cold[k][i]
        
        A = As
        B = Bs
        C = Cs
    
    return A, B, C
        
    
        
    

In [ ]:
df = pd.read_csv('../input/hotel-reviews/Datafiniti_Hotel_Reviews.csv', header = None, names = ['city', 'name', 'reviews.rating', 'reviews.userProvince'], usecols = [6,11,21,16])
#df=df.iloc[:5000, :]
#df['reviews.rating'] = pd.to_numeric(df["], downcast="float")

print('Dataset 1 shape: {}'.format(df.shape))
print('-Dataset examples-')
print(df.iloc[:5000, :])


In [ ]:
#Remove null values
df = df[df['reviews.userProvince'].notna()]

samples = 100

df1=df.sample(n = samples)
print(df1.shape)


df1.head()

    

In [ ]:
# Build The Tensor
df1['city']=df1['city'].astype('category').cat.codes
df1['name']=df1['name'].astype('category').cat.codes
df1['reviews.userProvince']=df1['reviews.userProvince'].astype('category').cat.codes
df1['reviews.rating']=df1['reviews.rating'].astype('category').cat.codes


l = list(df1['city'])
m = list(df1['name'])
n = list(df1['reviews.userProvince'])
#print(n)

city = dict([(y,x) for x,y in enumerate(set(l))])
name = dict([(y,x) for x,y in enumerate(set(m))])
province = dict([(y,x) for x,y in enumerate(set(n))])

all_values = city.values()
cityNo = max(all_values)

all_values = name.values()
nameNo = max(all_values)

all_values = province.values()
#print(all_values)
provinceNo = max(all_values)
#print(countryNo)


indexlist = []
valuelist = []
  

ex = np.zeros([cityNo+1,nameNo+1,provinceNo+1])


for i,j in df1.iterrows():
    #print(j.Movie_Id)
    x = city[df1['city'][i]]
    y = name[df1['name'][i]]
    z = province[int(df1['reviews.userProvince'][i])]

    #print(str(x)+" "+str(y)+" "+str(z))
    info = df1['reviews.rating'][i]
    
    #print(info)
    #print(x,y,z)
    ex[x][y][z] = info
        


In [ ]:

#initial inputs

# s specifies the number of samples to be selected in the tensor summary
# Higher the value of s, higher the accuracy
# But if s is too high, it may result in overfitting
s = [5,5,5]
rank = 3

# Parafac is applied on the initial tensor. This is the only time parafac is applied on the entire tensor
A = tensorly.tensor(ex)
factors = parafac(A,rank = rank, verbose = 0,normalize_factors=False)




# Selected random indices from mode 1,2 and 3 of initial tensor as the FIXED INDICES
# These samples will be used to estimate the factors later on when new slices are added instead of the full tensor
Xs,selectedI,selectedJ,selectedK,selectedindxI,selectedindxJ,selectedindxK = summary(A,s)

print("mode i selected:",selectedI.shape)
print("mode j selected:",selectedJ.shape)
print("mode k selected:",selectedK.shape)






In [ ]:
# The primary sambaten function
# First, a summary of the tensor is obtained with the previously selected samples and the new incomning slices
# Then this summary is factoriezed and the corresponding factors are used to estimate the original tensor
def SamBATen(X, rank, s, r):
    for i in range(0,r):
        if i==0:
            Xs,selectedI,selectedJ, selectedK, indI,indJ,indK =summary(X,s)
            fixed = []
            fixed.append(selectedI) 
            fixed.append(selectedJ) 
            fixed.append(selectedK)
            
        else:
            Xs,selectedI,selectedJ, selectedK, indI,indJ,indK =summary(X,s,fixed)
            
        factors = parafac(Xs,rank = rank, verbose = 0,normalize_factors=False)
        
        weights = factors[0]
        '''
        scaling_matrix = np.array(np.diag(weights**(1/3)))
        As= np.matmul(np.array(factors[1][0]),scaling_matrix)
        Bs= np.matmul(np.array(factors[1][1]),scaling_matrix)
        Cs= np.matmul(np.array(factors[1][2]),scaling_matrix)'''
        As= factors[1][0]
        Bs= factors[1][1]
        Cs= factors[1][2]
        
        
        
        if i ==0:
            A, B, C = merge(As,Bs,Cs,i,rank,selectedI,selectedJ, selectedK)
        else :
            A, B, C = merge(As,Bs,Cs,i,rank,selectedI,selectedJ, selectedK,Aold,Bold,Cold)
            
        Aold = A
        Bold = B
        Cold = C
    return A,B,C,weights
            

In [ ]:
# This function creates a new tensor by merging the new slices with the intial tensor
def merge_oldnew(A,F):
    m = A.shape[0]
    n = A.shape[1]
    p = A.shape[2]
    
    xx = F.shape[2]
    
    new = np.zeros(([m,n,p+xx]))
    
    fin = 0
    
    for i in range(0,m):
        for j in range(0,n):
            for k in range(0,new.shape[2]):
                if k < p:
                    new[i][j][k] = A[i][j][k]
                else:
                    new[i][j][k] = F[i][j][fin]
                    fin += 1
            fin = 0
            
    return new
    

In [ ]:
def insert_data(F,n):
    for i in range(0,F.shape[0]):
        for j in range(0,n):
            col = random.randrange(0,85)
            x = random.randrange(1,5)
            F[i][col][0] = x
    return F


In [ ]:
# SAMBATEN(original)
# Add new data repeatedly and apply incremental tensor decompostion for subsequent iterations
additions_no = 20
no_of_new_cols = 1
sambaten_error = 0
sambaten_fit = 0



Atotal = A

import tensorflow

r = 1
for i in range(0,additions_no):
    #Generate a random data sample as new data along the 3rd dimension
    F = 4*np.random.rand(Atotal.shape[0],Atotal.shape[1],no_of_new_cols)
    F = np.around(F, decimals=0)
    #print(X.shape)
    # The latest incoming slice is added to the total tensor(Atotal contains old and all new data)
    Atotal = merge_oldnew(Atotal,F)
    # Anew contains the selected initial sample slices and the last 10 new incoming slices
    Anew = buildXnew(Atotal,F,selectedI,selectedJ,selectedK)
    # Anew is used to estimate the new tensor instead of the entire tensor
    factor1,factor2,factor3,weights = SamBATen(Anew, rank, s, r)
    #print("Components on iteration ",i+1,":")
    #print(factor1,factor2,factor3)
    factor_array = []
    factor_array.append(factor1)
    factor_array.append(factor2)
    factor_array.append(factor3)
    
    #regenerated_tensor = regenerate(factors)
    regenerated_tensor = tensorly.cp_to_tensor((weights,factor_array))
    print("-------------------------------------------------------------------")
    print("Iteration no: ",i+1)
    residual=np.linalg.norm(Atotal) - np.linalg.norm(regenerated_tensor)
    fit = 1- ((residual) /np.linalg.norm(Atotal))
    print("fit",fit)
    error= abs(residual) /abs(np.linalg.norm(Atotal))
    print("error",error)
    sambaten_error += error
    sambaten_fit += fit
    print("------------------------------------------------------------------")





In [ ]:
# Compute the average error across all iterations

print("Original Sambaten error: ",(sambaten_error/additions_no)*100)
print("Original Sambaten fitness: ",(sambaten_fit/additions_no)*100)

In [ ]:
# SAMBATEN(modified)
# Add new data repeatedly and apply incremental tensor decompostion for subsequent iterations
additions_no = 20
no_of_new_cols = 1
sambaten_error = 0
sambaten_fit = 0



Atotal = A

import tensorflow

r = 1
for i in range(0,additions_no):
    #Generate a random data sample as new data along the 3rd dimension
    F = 4*np.random.rand(Atotal.shape[0],Atotal.shape[1],no_of_new_cols)
    F = np.around(F, decimals=0)
    #Depending on condition, we take the last 10 new incoming slices and merge them to our tensor summary
    if(i == 0):
        X = F
    else:
        X = merge_oldnew(X,F)
    if i >= 10:
        X = merge_oldnew(X,F)
        X = X[:,:,X.shape[2]-10:X.shape[2]]
    #print(X.shape)
    # The latest incoming slice is added to the total tensor(Atotal contains old and all new data)
    Atotal = merge_oldnew(Atotal,F)
    print("Atotal shape before:",Atotal.shape)
    print(i,Atotal.shape[2])
    Atotal = Atotal[:,:,1:Atotal.shape[2]]
    print("Atotal shape after:",Atotal.shape)
    # Anew contains the selected initial sample slices and the last 10 new incoming slices
    Anew = buildXnew(Atotal,X,selectedI,selectedJ,selectedK)
    # Anew is used to estimate the new tensor instead of the entire tensor
    factor1,factor2,factor3,weights = SamBATen(Anew, rank, s, r)
    #print("Components on iteration ",i+1,":")
    #print(factor1,factor2,factor3)
    factor_array = []
    factor_array.append(factor1)
    factor_array.append(factor2)
    factor_array.append(factor3)
    
    #regenerated_tensor = regenerate(factors)
    regenerated_tensor = tensorly.cp_to_tensor((weights,factor_array))
    print("-------------------------------------------------------------------")
    print("Iteration no: ",i+1)
    residual=np.linalg.norm(Atotal) - np.linalg.norm(regenerated_tensor)
    fit = 1- ((residual) /np.linalg.norm(Atotal))
    print("fit",fit)
    error= abs(residual) /abs(np.linalg.norm(Atotal))
    print("error",error)
    sambaten_error += error
    sambaten_fit += fit
    print("------------------------------------------------------------------")




In [ ]:
# Compute the average error across all iterations

print("Modified Sambaten error: ",(sambaten_error/additions_no)*100)
print("Modified Sambaten fitness: ",(sambaten_fit/additions_no)*100)

In [ ]:
# SAMBATEN(modified 2)
# Add new data repeatedly and apply incremental tensor decompostion for subsequent iterations
additions_no = 60
no_of_new_cols = 1
sambaten_error = 0
sambaten_fit = 0

# parameter
error_limit = 0.1
newSliceNo = 30



Atotal = A

import tensorflow

r = 1
for i in range(0,additions_no):
    #Generate a random data sample as new data along the 3rd dimension
    F = 4*np.random.rand(Atotal.shape[0],Atotal.shape[1],no_of_new_cols)
    F = np.around(F, decimals=0)
    #Depending on condition, we take the last 10 new incoming slices and merge them to our tensor summary
    if(i == 0):
        X = F
    elif i < 30:
        X = merge_oldnew(X,F)
    elif i >= 30:
        X = merge_oldnew(X,F)
        X = X[:,:,X.shape[2]-newSliceNo:X.shape[2]]
    #print(X.shape)
    # The latest incoming slice is added to the total tensor(Atotal contains old and all new data)
    Atotal = merge_oldnew(Atotal,F)
    print("Atotal shape before:",Atotal.shape)
    Atotal = Atotal[:,:,1:Atotal.shape[2]]
    print("Atotal shape after:",Atotal.shape)
    Anew = buildXnew(Atotal,X,selectedI,selectedJ,selectedK)
    # Anew is used to estimate the new tensor instead of the entire tensor
    factor1,factor2,factor3,weights = SamBATen(Anew, rank, s, r)
    #print("Components on iteration ",i+1,":")
    #print(factor1,factor2,factor3)
    factor_array = []
    factor_array.append(factor1)
    factor_array.append(factor2)
    factor_array.append(factor3)
    
    #regenerated_tensor = regenerate(factors)
    regenerated_tensor = tensorly.cp_to_tensor((weights,factor_array))
    print("-------------------------------------------------------------------")
    print("Iteration no: ",i+1)
    residual=np.linalg.norm(Atotal) - np.linalg.norm(regenerated_tensor)
    fit = 1- ((residual) /np.linalg.norm(Atotal))
    print("fit",fit)
    error= abs(residual) /abs(np.linalg.norm(Atotal))
    print("error",error)
    if error >= error_limit:
        print("Recalculate summary")
        Xs,selectedI,selectedJ,selectedK,selectedindxI,selectedindxJ,selectedindxK = summary(Atotal,s)
    sambaten_error += error
    sambaten_fit += fit
    print("------------------------------------------------------------------")

In [ ]:
# Compute the average error across all iterations

print("Modified 2 Sambaten error: ",(sambaten_error/additions_no)*100)
print("Modified 2 Sambaten fitness: ",(sambaten_fit/additions_no)*100)

Online CP, Tucker decomposition and Matrix Product State method are applied in the following cells to demonstrate how our method SAMBATEN performs compared to the existing tensor decomposition methods

**Online CP**

In [ ]:
# Online CP

A = tensorly.tensor(ex)

rank = 3



#Add new data repeatedly and apply incremental tensor decompostion for subsequent iterations
additions_no = 30
no_of_new_cols = 1

cp_error = 0
cp_fit = 0

m = A.shape[0]
n = A.shape[1]
p = A.shape[2]
Anew = A

r = 1
for i in range(0,additions_no):
    F = 4*np.random.rand(Anew.shape[0],Anew.shape[1],no_of_new_cols)
    F = np.around(F, decimals=0)
    Anew = merge_oldnew(Anew,F)
    cp_weight,cp_factors = parafac(Anew,rank = rank, verbose = 0,normalize_factors=False)
    cp_regenerated = tensorly.cp_to_tensor((cp_weight,cp_factors))
    print("-------------------------------------------------------------------")
    print("Iteration no: ",i+1)
    residual=np.linalg.norm(Anew) - np.linalg.norm(cp_regenerated)
    fit = 1- ((residual) /np.linalg.norm(Anew))
    print("fit",fit)
    error= abs(residual) /abs(np.linalg.norm(Anew))
    print("error",error)
    cp_error += error
    cp_fit += fit
    print("------------------------------------------------------------------")

In [ ]:
print("Online cp error: ",(cp_error/additions_no)*100)
print("Online cp fitness: ",(cp_fit/additions_no)*100)

**Matrix Product State Method**

In [ ]:
# Matrix Product State Method
A3 = tensorly.tensor(ex)

rank = 3

from tensorly.decomposition import matrix_product_state
from tensorly import tt_to_tensor



#Add new data repeatedly and apply incremental tensor decompostion for subsequent iterations
additions_no = 30
no_of_new_cols = 1

mat_product_error = 0
mat_product_fit = 0

m = A3.shape[0]
n = A3.shape[1]
p = A3.shape[2]
Anew3 = A3

r = 1
for i in range(0,additions_no):
    F = 4*np.random.rand(Anew3.shape[0],Anew3.shape[1],no_of_new_cols)
    Anew3 = merge_oldnew(Anew3,F)
    factors = matrix_product_state(Anew3, rank=rank)
    F = np.around(F, decimals=0)
    cp_regenerated3 = np.round(tt_to_tensor(factors))
    print("-------------------------------------------------------------------")
    print("Iteration no: ",i+1)
    residual=np.linalg.norm(Anew3) - np.linalg.norm(cp_regenerated3)
    fit = 1- ((residual) /np.linalg.norm(Anew3))
    print("fit",fit)
    error= abs(residual) /abs(np.linalg.norm(Anew3))
    print("error",error)
    mat_product_error += error
    mat_product_fit += fit

In [ ]:
print("Matrix Product State method error: ",(mat_product_error/additions_no)*100)
print("Matrix Product State method fitness: ",(mat_product_fit/additions_no)*100)

**Tucker Decomposition**

In [ ]:
# Tucker decomposition

A5 = tensorly.tensor(ex)

rank = 3

from tensorly.decomposition import matrix_product_state
from tensorly import tt_to_tensor
from tensorly.decomposition import tucker
from tensorly import tucker_to_tensor


#Add new data repeatedly and apply incremental tensor decompostion for subsequent iterations
additions_no = 30
no_of_new_cols = 1

tucker_error = 0
tucker_fit = 0

m = A5.shape[0]
n = A5.shape[1]
p = A5.shape[2]
Anew5 = A5

r = 1
for i in range(0,additions_no):
    F = 4*np.random.rand(Anew5.shape[0],Anew5.shape[1],no_of_new_cols)
    F = np.around(F, decimals=0)
    Anew5 = merge_oldnew(Anew5,F)
    
    factors = tucker(Anew5, rank=3)
    regenerated5 = tucker_to_tensor(factors)    
    print("-------------------------------------------------------------------")
    print("Iteration no: ",i+1)
    residual=np.linalg.norm(Anew5) - np.linalg.norm(regenerated5)
    fit = 1- ((residual) /np.linalg.norm(Anew5))
    print("fit",fit)
    error= abs(residual) /abs(np.linalg.norm(Anew5))
    print("error",error)
    tucker_error += error
    tucker_fit += fit

In [ ]:
print("Tucker method error: ",(tucker_error/additions_no)*100)
print("Tucker method fitness: ",(tucker_fit/additions_no)*100)

This we can see that all the other decomposition methods had much higher percentage of error and lower fitness percentage compared to SAMBATEN.